In [1]:
"""
RNN : 이미지를 막 쪼개서, 
INPUT DATA가 시계열적 특성 갖고 있을 때 활용할 수 있는 레이어
오퍼레이션담당의 유닛에 따라 종류가 나뉨.
LSTM - 바닐라RNN은, X데이터가 들어오면 행렬 W가 곱해지고, 이전 STATE에서의 결과물인
Ht-1을 행렬 U와 곱하고 bias를 더해서 hyperbolic탄젠트 취한 것을 아웃풋으로 뽑거나,
다음 레이어의 인풋으로 들여보냄
여기서 LSTM은 더 나아가, 과거의 값이 최종값에 영향을 주기 힘든 기존 바닐라를 개선해,
INPUT을 받아 다음 레이어의 OPERATION으로 바로 보내는 길을 만듬
"""

SyntaxError: ignored

In [12]:
import tensorflow as tf
from keras.models import Model
from keras.layers import Input, Dense, LSTM
import numpy as np
from tensorflow import keras
from tensorflow.keras import layers

x = np.array([[[1.], [2.], [3.], [4.], [5.]], [[2.], [3.], [4.], [5.], [6.]],
              [[3.],[4.],[5.],[6.],[7.]]])
y = np.array([[6.],[7.],[8.]])

In [8]:
x.shape, y.shape # 차원. 하나의 데이터포인트가 가지는 개수가 5
#총 3개의 12345 23456 34567, 각각 5개이며, feautur demand는 1. 각 [1.]에 든 것 개수
#만약 이미지데이터라면, [1., 0., 1...]일 것이다. 이 경우엔 1이 아닐것

((3, 5, 1), (3, 1))

In [13]:
inputs = Input(batch_shape = (None, 5, 1))
h = LSTM(3)(inputs)
"""
LSTM레이어 정의에, 앞서말한 5가 쓰임. 위의 INPUT은 X를 받아오는데, 
이 SHAPE은, 5라 정의 되어있어야, LSTM이 이를 받아올 때 5번계산을해야 함을 알수있음
5,1을 넘겨주게 되면, LSTM을 생성. KERAS LAYER밑에 정의되어 있으며, 
(3)은, 아웃풋으로 내보낼 FEATURE DIMENSION. F통해 계산한 결과가 H인데, 이 H에 해당하는
SHAPE이 3으로 정의시킨 것. 이는 정의하기 나름이여, 결과 Output shape는 model.summary()
통해 알 수있을 것. 모델 경량화를 위한다면 이 모델을 줄이라.
output dimension은 1이어야 한다. y의 shape이, 출력해야하는 값이 [6.]이나 [7.]형태로
하나씩 들어있기에, dimension값은 1이어야 한다. 
따라서, 최종적으로 Dense의 첫번째 인자는 1.
물론 아래 아웃풋을 없애고, outputs = LSTM(1)(inputs)로 해도 되지만, fully connected를
한번더 실행했다.
아담 옵티파이저, mean_squared_error사용.
최종 파라미터는 64개 등장.
"""
outputs = Dense(1, activation = 'relu')(h)

model = Model(inputs, outputs)
opt = keras.optimizers.Adam(learning_rate = 0.01)
model.compile(loss = 'mean_squared_error', optimizer = opt)

model.summary()

Model: "model_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         [(None, 5, 1)]            0         
_________________________________________________________________
lstm_5 (LSTM)                (None, 3)                 60        
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 4         
Total params: 64
Trainable params: 64
Non-trainable params: 0
_________________________________________________________________


In [14]:
model.fit(x,y,epochs=200, batch_size = 1)

Epoch 1/200
3/3 [==============================] - 2s 6ms/step - loss: 45.3951
Epoch 2/200
3/3 [==============================] - 0s 5ms/step - loss: 41.9492
Epoch 3/200
3/3 [==============================] - 0s 5ms/step - loss: 40.0074
Epoch 4/200
3/3 [==============================] - 0s 5ms/step - loss: 38.3069
Epoch 5/200
3/3 [==============================] - 0s 5ms/step - loss: 36.6396
Epoch 6/200
3/3 [==============================] - 0s 6ms/step - loss: 35.1058
Epoch 7/200
3/3 [==============================] - 0s 5ms/step - loss: 33.6472
Epoch 8/200
3/3 [==============================] - 0s 6ms/step - loss: 32.3331
Epoch 9/200
3/3 [==============================] - 0s 7ms/step - loss: 30.9988
Epoch 10/200
3/3 [==============================] - 0s 5ms/step - loss: 29.7157
Epoch 11/200
3/3 [==============================] - 0s 5ms/step - loss: 28.5679
Epoch 12/200
3/3 [==============================] - 0s 5ms/step - loss: 27.4954
Epoch 13/200
3/3 [==============================]

In [15]:
# 위의 결과를 보면, loss가 굉장히 감소함
model.predict(x)
#predict값은, 6, 7, 8에 어느정도 근접한 것을 알 수 있다.
#시계열정도에 따라 영향주는 정도를 알고싶다? 과거 2개의 스텝을 더 위주로 봐서 최종결과 내기 등
"""
시계열적 특성 반영코자 할때 RNN을 쓴다.
보통 언어모델에 쓰임.
"""

array([[5.88652  ],
       [7.1066732],
       [7.7134056]], dtype=float32)